In [116]:
import torch

# Tensors

Contiguous memory

In [117]:
torch.random.manual_seed(1)
x = torch.rand((2, 4, 3), dtype=torch.float32)


print(x)
print("-"*100)
print(x.flatten())
print("-"*100)
print("Shape:", x.shape)
print("Stride:", x.stride())
print(x.is_contiguous())


tensor([[[0.7576, 0.2793, 0.4031],
         [0.7347, 0.0293, 0.7999],
         [0.3971, 0.7544, 0.5695],
         [0.4388, 0.6387, 0.5247]],

        [[0.6826, 0.3051, 0.4635],
         [0.4550, 0.5725, 0.4980],
         [0.9371, 0.6556, 0.3138],
         [0.1980, 0.4162, 0.2843]]])
----------------------------------------------------------------------------------------------------
tensor([0.7576, 0.2793, 0.4031, 0.7347, 0.0293, 0.7999, 0.3971, 0.7544, 0.5695,
        0.4388, 0.6387, 0.5247, 0.6826, 0.3051, 0.4635, 0.4550, 0.5725, 0.4980,
        0.9371, 0.6556, 0.3138, 0.1980, 0.4162, 0.2843])
----------------------------------------------------------------------------------------------------
Shape: torch.Size([2, 4, 3])
Stride: (12, 3, 1)
True


View vs copy

In [118]:
y_view = x[0]
print(y_view.shape)
print(y_view.stride())
print(y_view.is_contiguous())

torch.Size([4, 3])
(3, 1)
True


Not all views are contiguous

In [119]:
torch.random.manual_seed(1)
x = torch.rand((2, 4, 3), dtype=torch.float32)

y = x.transpose(1, 2)

print(y)

print("-"*100)

print(y.shape)
print(y.stride())
print(y.is_contiguous())
print("-"*100)

tensor([[[0.7576, 0.7347, 0.3971, 0.4388],
         [0.2793, 0.0293, 0.7544, 0.6387],
         [0.4031, 0.7999, 0.5695, 0.5247]],

        [[0.6826, 0.4550, 0.9371, 0.1980],
         [0.3051, 0.5725, 0.6556, 0.4162],
         [0.4635, 0.4980, 0.3138, 0.2843]]])
----------------------------------------------------------------------------------------------------
torch.Size([2, 3, 4])
(12, 1, 3)
False
----------------------------------------------------------------------------------------------------


In [120]:
y = x[:,0].clone()
print(y.shape)
print(y.stride())
print(y.is_contiguous())

torch.Size([2, 3])
(3, 1)
True


.view(), .expand(), .unsqueeze()/.squeeze(), .transpose(), .permute()

In [121]:
torch.random.manual_seed(1)
x = torch.randn(2, 4, 3)
x.shape

torch.Size([2, 4, 3])

In [122]:
y_reshape = x.view(8, 3)
print(y_reshape.shape)
print(y_reshape.is_contiguous())
print(y_reshape.stride())

torch.Size([8, 3])
True
(3, 1)


In [123]:
y = x.transpose(0,1)
print(y.is_contiguous())

try:
  y.view(8, 3)
except RuntimeError as e:
  print(f"RuntimeError: {e}")

False
RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.


In [124]:
torch.random.manual_seed(1)
x = torch.randn(2, 4, 3)
x.shape

torch.Size([2, 4, 3])

In [125]:
y_exp = x.unsqueeze(1)
print(y_exp.shape)
y_exp = y_exp.expand(2, 1_000_000_000, 4, 3)
print(y_exp.shape)
print(y_exp.stride())
print(y_exp.is_contiguous())
# print(y_exp.stride())

torch.Size([2, 1, 4, 3])
torch.Size([2, 1000000000, 4, 3])
(12, 0, 3, 1)
False


.view() vs .reshape()

In [126]:
try:
    y_exp.view(2_000_000_000, 4, 3)
except RuntimeError as e:
    print(f"RuntimeError: {e}")

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.


In [127]:
# try:
#     y_exp.reshape(2_000_000_000, 4,3)
# except RuntimeError as e:
#     print(f"RuntimeError: {e}")


cuda

In [128]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [129]:
device == 'cuda'

False

In [130]:
import time

x = torch.randn(5000, 5000)
start = time.time()
y = x @ x


print("CPU time:", time.time() - start)

if device == 'cuda':
    x = x.to(device)
    start = time.time()
    y = x @ x
    torch.cuda.synchronize()
    print("GPU time:", time.time() - start)


CPU time: 0.08591103553771973


- torch.nn.Parameter: model-tracked tensor with gradients (used in all neural nets)
- TensorDict: batched dictionary of tensors (ideal for structured data, RL, dynamic models)
- LazyTensor: symbolic tensor used in PyTorch 2.x compiler stack (torch.compile, tracing)
- PackedSequence: handles variable-length sequences in RNNs (efficient batching)
- Meta tensor: zero-allocation tensor for shape checking or debugging models without memory usage


# Automatic differentiation

Let $f_i(z_{i-1}; w_{i})$ be some function for $i \in [n].$ Define
$$
L(x;w) = f_n(f_{n-1}(\cdots f_1(x; w_1) \cdots; w_{n-1}); w_n)
$$
We want to compute the derivatives $\frac{\partial L}{\partial w_i}(x,w),$ $i = 1,\dots,n$.

Define intermediate variables:
$$
z_0 = x, \quad z_i = f_i(z_{i-1}; w_i) \text{ for } i = 1, \ldots, n, \quad L = z_n.
$$

Backward propagation computes:
$$
\delta_n = \frac{\partial L}{\partial z_n} = 1
$$

and for $i = n,\dots, 1$
$$
\frac{\partial L}{\partial w_i} = \delta_i \cdot \frac{\partial f_i}{\partial w_i}(z_{i-1},w_i).
$$

$$
\delta_{i-1} = \delta_i \cdot \frac{\partial f_i}{\partial z_{i-1}}(z_{i-1},w_i),
$$



In [131]:
torch.random.manual_seed(1)

x = torch.rand(3, requires_grad=True)

loss = torch.logsumexp(x, dim = 0)

loss.backward()
print("dz/dx =", x.grad)
print('softmax', torch.softmax(x, 0))

dz/dx = tensor([0.4308, 0.2670, 0.3022])
softmax tensor([0.4308, 0.2670, 0.3022], grad_fn=<SoftmaxBackward0>)


Computational graph, .is_leaf()

$z =((x *y) + sin(x))^2$

In [132]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Intermediate values
a = x * y
b = torch.sin(x)
c = a + b
z = c**2

z.backward()

print("dL/dx =", x.grad)
print("dL/dy =", y.grad)

dL/dx = tensor(35.7052)
dL/dy = tensor(27.6372)


In [133]:
z.grad_fn

In [134]:
def print_graph(fn, indent=0):
    if fn is None:
        return
    print(" " * indent, f"{type(fn).__name__}")
    for next_fn, _ in fn.next_functions:
        print_graph(next_fn, indent + 2)

print_graph(z.grad_fn)


 PowBackward0
   AddBackward0
     MulBackward0
       AccumulateGrad
       AccumulateGrad
     SinBackward0
       AccumulateGrad


In [135]:
#!pip install torchviz

In [136]:
# import torch
# from torchviz import make_dot
# from IPython.display import Image

# # Step 4: Visualize the graph
# dot = make_dot(z, params={'x': x, 'y': y, 'a': a, 'b': b, 'c': c, 'z': z})
# dot.format = "png"
# dot.render("comp_graph", cleanup=True)
# Image("comp_graph.png")


Gradients accumulate

In [137]:
torch.manual_seed(342)

x = torch.rand(3, requires_grad=True)

# First backward
loss1 = torch.logsumexp(x, dim= 0)
loss1.backward()

print("✅ After first backward:")
print("x.grad =", x.grad)

# Second backward without zeroing the gradient
loss2 = x.sum()
x.grad.zero_()
# loss2 = torch.logsumexp(x, dim= 0)
loss2.backward()

print("\n⛔ After second backward (without zeroing):")
print("x.grad =", x.grad)


✅ After first backward:
x.grad = tensor([0.3760, 0.2959, 0.3281])

⛔ After second backward (without zeroing):
x.grad = tensor([1., 1., 1.])


jacobian, grad

In [138]:
from torch.autograd.functional import jacobian
from torch.autograd import grad

torch.manual_seed(1)
x = torch.rand(3, requires_grad=True)

def f(x):
    return torch.softmax(x, 0)

J = jacobian(f, x)
print("Jacobian:")
print(J)

def g(x):
  return torch.logsumexp(x, 0)


print("\nGradient:")
print(grad(g(x), x)[0])
# print(x.grad)

Jacobian:
tensor([[ 0.2452, -0.1150, -0.1302],
        [-0.1150,  0.1957, -0.0807],
        [-0.1302, -0.0807,  0.2109]])

Gradient:
tensor([0.4308, 0.2670, 0.3022])


In [139]:
torch.manual_seed(1)
x = torch.rand(3, requires_grad=True)

y = torch.logsumexp(x, 0)
print("✅ First: Compute gradient with autograd.grad")
print("grad(y, x):", grad(y, x)[0])


print("\n⛔ Then: Try backward() on the same graph")
try:
    y.backward()
except RuntimeError as e:
    print(f"RuntimeError: {e}")


print("\n✅ Finally: Recompute y and call backward() again")
y = torch.logsumexp(x, 0)
y.backward()
print("x.grad:", x.grad)

✅ First: Compute gradient with autograd.grad
grad(y, x): tensor([0.4308, 0.2670, 0.3022])

⛔ Then: Try backward() on the same graph
RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

✅ Finally: Recompute y and call backward() again
x.grad: tensor([0.4308, 0.2670, 0.3022])


In [140]:
x = torch.tensor(2.0, requires_grad=True)
y = x**3
dy_dx = grad(y, x, create_graph=True)[0]
d2y_dx2 = grad(dy_dx, x)[0]
print(d2y_dx2)


tensor(12.)


Clones share the computational graph with the original variable

In [141]:
x = torch.rand(5, requires_grad=True)

x_copy = x.clone()  #.detach().requires_grad_() # Not detached!
y = (x_copy).norm()
y.backward()

print("x.grad:", x.grad)
print("x_copy.is_leaf:", x_copy.is_leaf)
print("x_copy.grad:", x_copy.grad)

x.grad: tensor([0.5320, 0.0212, 0.5792, 0.2876, 0.5462])
x_copy.is_leaf: False
x_copy.grad: None


/var/folders/sh/7ch7dbpn0db2smxkj3s5hbf80000gn/T/ipykernel_20429/3066404.py:9: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  print("x_copy.grad:", x_copy.grad)


In [142]:
x = torch.rand((3,4), requires_grad= True)
x_copy = x.clone().detach().requires_grad_()


# ❌ PyTorch throws here because in-place ops on leaf tensors that require gradients are dangerous. If it allowed this, it would overwrite values that autograd needs.
try:
  x += 1
except RuntimeError as e:
  print(f"RuntimeError: {e}")

print('Gradient at x: ', x.grad)
print("-"*100)

# ✅ This is NOT in-place. But now... x is no longer a leaf tensor.
x = x + 1
print('x requires grad?: ', x.requires_grad)
print('x is a leaf tensor?: ', x.is_leaf)

x.retain_grad()

y = x.norm()
y.backward()
print(x.grad)

print("-"*100)


with torch.no_grad():
  x_copy += 1
y = x_copy.norm()
y.backward()

print(x_copy.grad)


RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.
Gradient at x:  None
----------------------------------------------------------------------------------------------------
x requires grad?:  True
x is a leaf tensor?:  False
tensor([[0.2887, 0.2647, 0.3015, 0.2805],
        [0.3095, 0.2401, 0.2692, 0.2677],
        [0.2893, 0.2756, 0.3564, 0.3046]])
----------------------------------------------------------------------------------------------------
tensor([[0.2887, 0.2647, 0.3015, 0.2805],
        [0.3095, 0.2401, 0.2692, 0.2677],
        [0.2893, 0.2756, 0.3564, 0.3046]])


In [143]:
try:
  x = torch.rand((3,4), requires_grad= True)
  # y = x[:2]
  # y += 1
  x += 1
except RuntimeError as e:
  print(f"RuntimeError: {e}")

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.


# Optimization

In [ ]:
num_agents = 1000
num_items = 2
num_features = 3


torch.manual_seed(1)
x_i_j_k = torch.normal(0,1,(num_agents, num_items, num_features))
beta0_k = torch.randint(-10,10, size = (num_features,), dtype= x_i_j_k.dtype)
# beta0_k = torch.ones(num_features, dtype=x_i_j_k.dtype) 

U_i_j = x_i_j_k @ beta0_k
mu_i_j = torch.softmax(U_i_j, dim= 1)
j_i = torch.multinomial(mu_i_j, 1)

In [145]:
Lip_ll =  2 * x_i_j_k.norm(dim= 2).max(1).values.sum()

learning_rate = 1.9 / Lip_ll
print(learning_rate)

tensor(0.0005)


$$- ll = - \log \prod_{i \ in I} \frac{\exp(X_{i j_i}^\top \beta)}{\sum_j \exp(X_{i j}^\top \beta)} = \sum_i \log \sum_j \exp(X_{i j}^\top \beta)  - \sum_i X_{i j_i}^\top \beta$$

In [146]:
def neg_log_likelihood(beta_k, x_i_j_k, j_i):

    U_i_j = x_i_j_k @ beta_k
    U_i_hat = U_i_j.gather(1, j_i)

    return torch.logsumexp(U_i_j, dim=1).sum() - U_i_hat.sum()

def compute_MLE_diy(x_i_j_k, j_i, tol=1e-4, lr=1e-2, num_epochs = 1000):

    _, _, num_features = x_i_j_k.shape

    torch.manual_seed(2)
    beta_k = torch.randn(num_features, requires_grad=True)


    for epoch in range(num_epochs):
        if beta_k.grad is not None:
            beta_k.grad.zero_()

        loss = neg_log_likelihood(beta_k, x_i_j_k, j_i)
        loss.backward()

        grad_norm = beta_k.grad.norm()
        if grad_norm < tol:
            print(f"Converged at tolerance:", tol)
            break

        with torch.no_grad():
            beta_k -= lr * beta_k.grad

    return beta_k

def compute_MLE(x_i_j_k, j_i, tol = 1e-4, lr = 1e-2, num_epochs = 1000):

    _,_,num_features = x_i_j_k.shape
    beta_k = torch.randn(num_features, requires_grad=True)

    optimizer = torch.optim.SGD([beta_k], lr= lr)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        loss = neg_log_likelihood(beta_k, x_i_j_k, j_i)
        loss.backward()

        if beta_k.grad.norm() < tol:
            print(f"Converged at tolerance:", tol)
            break
        optimizer.step()

    return beta_k

In [147]:
num_epochs = 5000
lr = learning_rate

tic = time.time()
beta_hat = compute_MLE(x_i_j_k, j_i, lr= lr, num_epochs = num_epochs)
print(beta_hat)
print(beta0_k)
print("Time:", time.time() - tic)

tensor([ 3.5353, -0.2296, -8.6008], requires_grad=True)
tensor([ 3.,  0., -7.])
Time: 0.4663581848144531


In [148]:
from torch.autograd.functional import hessian

H_k_k = hessian(lambda beta_k: neg_log_likelihood(beta_k, x_i_j_k, j_i), beta_hat)
Avar_k_k = torch.linalg.inv(H_k_k)
print("Asymptotic variance:")
print(Avar_k_k)
print("Standard errors:", torch.sqrt(torch.diag(Avar_k_k)))

Asymptotic variance:
tensor([[ 0.1541, -0.0083, -0.3318],
        [-0.0083,  0.0179,  0.0198],
        [-0.3318,  0.0198,  0.7962]])
Standard errors: tensor([0.3926, 0.1336, 0.8923])


# Neural Network

In [149]:
from torch.utils.data import TensorDataset, DataLoader, random_split

num_agents = 1000
num_items = 10
num_features = 3


torch.manual_seed(1)
x_i_j_k = torch.normal(0,1,(num_agents, num_items, num_features))
beta0_k = torch.randint(-10,10, size = (num_features,), dtype= x_i_j_k.dtype)

U_i_j = x_i_j_k @ beta0_k
mu_i_j = torch.softmax(U_i_j, dim = 1)
j_i = torch.multinomial(mu_i_j, 1)

dataset = TensorDataset(x_i_j_k, j_i)
# loader = DataLoader(dataset, batch_size=128, shuffle=True)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

x_train, j_train = dataset[train_dataset.indices]
x_test, j_test = dataset[test_dataset.indices]

In [150]:
import torch.nn as nn

class LogitNet(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.logit_model = nn.Linear(num_features, 1, bias=False)

    def forward(self, x_i_j_k):
        I, J, K = x_i_j_k.shape
        x_ij_k = x_i_j_k.view(I * J, K)

        U_i_j = self.logit_model(x_ij_k).view(I, J)
        return U_i_j

logit_example = LogitNet(num_features)
optimizer = torch.optim.SGD(logit_example.parameters(), lr=1e-2)
torch.manual_seed(2)
loss_fn = nn.CrossEntropyLoss(reduction='sum')

num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    U_i_j = logit_example(x_train)
    loss = loss_fn(U_i_j, j_train.squeeze(1))
    loss.backward()
    optimizer.step()

if num_features< 5:
    with torch.no_grad():
        beta_net = logit_example.logit_model.weight.squeeze()
        print(beta_net)
        print(beta0_k)

tensor([-8.1355,  7.0501,  0.1358], requires_grad=True)
tensor([-8.,  7.,  0.])


In [151]:
class LogitNet3Layer(nn.Module):
    def __init__(self, num_features, hidden_dim=64):
        super().__init__()
        self.fc1 = nn.Linear(num_features, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, x_i_j_k):
        I, J, K = x_i_j_k.shape
        x = x_i_j_k.view(I * J, K)

        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        U_i_j = self.output(x).view(I, J)

        return U_i_j


nn_example = LogitNet3Layer(num_features, hidden_dim = 64)
optimizer = torch.optim.SGD(nn_example.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss(reduction='sum')

num_epochs = 1000

for epoch in range(num_epochs):

    optimizer.zero_grad()
    U_i_j = nn_example(x_train)
    loss = loss_fn(U_i_j, j_train.squeeze(1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(nn_example.parameters(), max_norm=1.0)
    optimizer.step()

In [152]:
def compute_accuracy(model, x_train, j_train):
  model.eval()
  logits = model(x_train)
  return (logits.argmax(dim=1) == j_train.squeeze(1)).float().mean()


acc_logit_train = compute_accuracy(logit_example, x_train, j_train)
acc_logit_test = compute_accuracy(logit_example, x_test, j_test)

acc_nn_train = compute_accuracy(nn_example, x_train, j_train)
acc_nn_test = compute_accuracy(nn_example, x_test, j_test)

print(f"LogitNet  - Train Acc: {acc_logit_train:.3f}, Test Acc: {acc_logit_test:.3f}")
print(f"3LayerNet - Train Acc: {acc_nn_train:.3f}, Test Acc: {acc_nn_test:.3f}")

LogitNet  - Train Acc: 0.897, Test Acc: 0.915
3LayerNet - Train Acc: 0.919, Test Acc: 0.900
